<a href="https://colab.research.google.com/github/sriksmachi/sriksml/blob/main/ResamplingApproaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resampling Approaches

- Validation set (Baseline)
- Leave one out sampling
- K-fold cross validation

In [1]:
import numpy as np
